In [2]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from transformers import AutoTokenizer
from pathlib import Path

model_name = "digit82/kobart-summarization"
onnx_dir = Path("../../automation/models/kobart_summary_onnx")

# ✅ 1. 모델 변환 (export=True) ← 저장 안됨
model = ORTModelForSeq2SeqLM.from_pretrained(model_name, export=True)

# ✅ 2. ONNX 모델 저장
model.save_pretrained(onnx_dir)

# ✅ 3. 토크나이저 저장
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(onnx_dir)

print("🎉 ONNX 변환 및 저장 완료:", onnx_dir)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
c:\Users\user\anaconda3\envs\test-0602\lib\site-packages\transformers\models\bart\

🎉 ONNX 변환 및 저장 완료: ..\..\automation\models\kobart_summary_onnx


In [77]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from tokenizers import Tokenizer
from pathlib import Path
import torch

def get_summarize_model():
    """
    ONNX 모델과 토크나이저를 반환하는 함수
    """

    model_dir = Path("../../automation/models/kobart_summary_onnx")
    model_summarize = ORTModelForSeq2SeqLM.from_pretrained(model_dir, local_files_only=True)

    tokenizer_summarize = Tokenizer.from_file(str(model_dir / "tokenizer.json"))

    return model_summarize, tokenizer_summarize

In [78]:
model_summarize, tokenizer_summarize = get_summarize_model()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [81]:
def summarize_event_focused(text, model_summarize, tokenizer_summarize):
    # 토큰화
    encoding = tokenizer_summarize.encode(text)
    input_ids = torch.tensor([encoding.ids])

    text_length = len(text)

    # 1. 최소 길이는 짧은 본문은 고정, 긴 본문은 점진적으로 증가
    def compute_min_length(text_length: int) -> int:
        if text_length < 300:
            return 30
        else:
            return 50

    min_len = compute_min_length(text_length)
    max_len = round(text_length * 0.5) + 50  # 더 여유를 주되 max 길이 제한

    # 생성
    summary_ids = model_summarize.generate(
        input_ids,
        min_length=min_len,  # 최소 요약 길이 (예: 30)
        max_new_tokens=max_len,  # 추가 생성 최대 토큰 수
        num_beams=4,  # 빔 서치 개수
        length_penalty=1.0,  # 너무 긴 문장 방지
        repetition_penalty=1.3,  # 반복 억제
        no_repeat_ngram_size=3,  # 동일한 3-gram 반복 금지
        early_stopping=True,  # <eos> 토큰 생성 시 중단
        # do_sample은 사용하지 않음 (ONNX 미지원)
    )
    summary = tokenizer_summarize.decode(summary_ids[0].tolist(), skip_special_tokens=True)

    return summary

In [82]:
# 입력 문장
text = """
카카오페이, 신세계 페이 품나 이커머스 결제 강화 점유 확대 압도적 1위 네이버에 도전장 매각가 5천억원 안팎 달할 듯 카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 간편결제 시장에서 영향력을 확대하기 위한 차원이다. 미리 계좌·카드를 등록만 해놓으면 간편하게 물건·서비스를 결제할 수 있는 간편결제 시장은 해마다 가파른 성장세를 보이며 전자금융 업계의 중요한 먹거리로 급부상 중이다. 23일 한국은행 등에 따르면 국내 간편지급(간편결제) 서비스 시장의 일평균 이용 금액은 지난해 기준 9594억원에 달한다. 이 중 네이버페이, 카카오페이, 토스페이 등 전자금융업자가 일평균 4814억원, 휴대폰 제조 회사(삼성페이·애플페이 등)가 일평균 2443억원, 금융회사(카드사·은행 등)가 일평균 2337억원을 담당하고 있다. 전자금융업자가 간편지급 시장의 50.7%를 차지하고 있는 셈이다. 현재 전자금융업자는 총 40개에 달하는데, 이 중 빅3인 네이버페이, 카카오페이, 토스페이가 90%의 점유율을 기록 중이다. 이 밖에 쓱페이, 당근페이, 무신사페이, 컬리페이 등도 경쟁을 펼치고 있다. 카카오페이는 선물하기 등 결제 기능이 있지만, 이커머스와 연동돼 있는 네이버, 카드사와 연동돼 있는 토스에 비해 시장점유율이 낮은 상황이다. 이 때문에 카카오페이는 이커머스 기능이 있는 쓱페이·스마일페이 인수를 시도하며 시장점유율 확대를 노릴 것으로 전망된다. 쓱페이와 스마일페이는 주 이용률 빈도는 낮지만, 이용자가 약 2500만명에 달해 카카오페이와 시너지 효과를 극대화할 수 있다. 앞서 신세계그룹은 지난해 한 차례 쓱페이·스마일페이에 대한 매각 작업을 추진하며 우선협상대상자로 토스(비바리퍼블리카)를 선정한 바 있다. 당시 거론된 가격은 7000억원 선이었다. 하지만 양측은 이후 시너지 창출 방안과 관련해 이견이 생겼고, 결국 매각 작업이 무산됐다. 이런 가운데 이날 이마트의 자회사 SSG닷컴은 간편결제 사업부인 \'쓱페이\'를 물적분할해 신설 회사인 \'플래티넘페이먼츠\'를 설립한다고 밝혔다. 분할 기일은 2025년 7월 1일이며, 신설 법인은 SSG닷컴 자회사로 편입될 예정이다. SSG닷컴 측은 "간편결제 서비스의 전문성을 강화하고, 범용성을 높여 경영 효율성을 제고하기 위한 결정"이라고 밝혔다. 관련 업계에서는 간편결제 사업 매각을 위한 이마트 측의 사전정지 작업으로 해석하고 있다. 현재 투자은행(IB) 업계에서 거론되는 쓱페이·스마일페이의 매각가는 약 5000억원 안팎이다. 이번 매각으로 자금이 유입되면 이마트의 재무 개선에도 도움이 될 전망이다. 이마트의 부채 비율은 올해 3월 말 기준 117.4%로, 2022~2023년 90%대 초반이었던 것과 비교하면 부채가 늘어난 상황이다
"""
summary = summarize_event_focused(text, model_summarize, tokenizer_summarize)

print("📄 요약:", summary)

📄 요약: 카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 간편결제 시장에서 영향력을 확대하기 위한 차원이며, 쓱페이와 스마일페이는 주 이용률 빈도는 낮지만, 이용자가 약 2500만명에 달해 카카오페이와 시너지 효과를 극대화할 수 있다.


In [29]:
import pandas as pd

# 행 전체 출력
pd.set_option("display.max_rows", None)

# 열 전체 출력
pd.set_option("display.max_columns", 200)

# 각 열 너비 무제한 출력
pd.set_option("display.max_colwidth", 200)

# 전체 넓이 제한 해제
pd.set_option("display.width", None)

In [18]:
import re
from kss import split_sentences

def remove_market_related_sentences(text: str) -> str:
    # 줄바꿈 제거
    text = text.replace("\n", " ")

    # 대괄호 포함 텍스트 제거: [파이낸셜뉴스], [사진] 등
    text = re.sub(r"\[[^\]]*\]", "", text)

    # '/사진', '/사진제공' 제거
    text = re.sub(r"/사진(제공)?", "", text)

    # 이메일 주소 제거 (예: josh@yna.co.kr)
    text = re.sub(r"\b[\w.-]+@[\w.-]+\.\w+\b", "", text)

    # 문장 단위 분리 (간단하게 마침표 기준, 필요시 KSS 등 적용 가능)
    sentences = split_sentences(text)

    # 제거할 패턴들 (뉴스 문장에서 자주 등장하는 패턴)
    patterns = [
        r"(자세한 내용|자세한 사항)",  # 뉴스 기본 표현
        r"\d{4}[.-]\d{1,2}[.-]\d{1,2}",  # 날짜 (예: 2025.03.26, 2024-12-01)
        r"([0-9,]+(?:만)?[0-9,]*\s?(?:원|만원))",  # 가격 (예: 3,500원, 12000원)
        r"(강세|펀드|시가총액|등락률|한국거래소)",  # 증시 용어
        r"\([+-]?[0-9.,]+%\)",  # 괄호 안 퍼센트 등락률
        r"(투자의견|연구원|평가|예상치|증권가|리포트|팀장)",  # 애널리스트 용어
        r"(순이익|전년|매출|영업이익|영업적자|증시|코스피|코스닥|다우|나스닥|매출액|거래일|호조세|레버리지|투자자|조정|자산|수익률|이익률|수익성|내리막|부진한|낙폭|기대치|실적발표|기업 가치)",  # 시장 용어
    ]

    # 하나의 통합 패턴으로 컴파일
    combined_pattern = re.compile("|".join(patterns))

    # 필터링된 문장만 유지
    filtered = [s for s in sentences if not combined_pattern.search(s)]

    text_preprocessed = " ".join(filtered)

    # print(f"원문:{sentences}\n|\n전처리 된 문장: {text_preprocessed}\n\n")

    return text_preprocessed

In [19]:
df_5 = df.head(5).copy()
df_5['article'] = df_5['article'].apply(remove_market_related_sentences)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [36]:
df_5['article'][4]

'카카오페이, 신세계 페이 품나 이커머스 결제 강화 점유 확대 압도적 1위 네이버에 도전장 매각가 5천억원 안팎 달할 듯 카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 간편결제 시장에서 영향력을 확대하기 위한 차원이다. 미리 계좌·카드를 등록만 해놓으면 간편하게 물건·서비스를 결제할 수 있는 간편결제 시장은 해마다 가파른 성장세를 보이며 전자금융 업계의 중요한 먹거리로 급부상 중이다. 23일 한국은행 등에 따르면 국내 간편지급(간편결제) 서비스 시장의 일평균 이용 금액은 지난해 기준 9594억원에 달한다. 이 중 네이버페이, 카카오페이, 토스페이 등 전자금융업자가 일평균 4814억원, 휴대폰 제조 회사(삼성페이·애플페이 등)가 일평균 2443억원, 금융회사(카드사·은행 등)가 일평균 2337억원을 담당하고 있다. 전자금융업자가 간편지급 시장의 50.7%를 차지하고 있는 셈이다. 현재 전자금융업자는 총 40개에 달하는데, 이 중 빅3인 네이버페이, 카카오페이, 토스페이가 90%의 점유율을 기록 중이다. 이 밖에 쓱페이, 당근페이, 무신사페이, 컬리페이 등도 경쟁을 펼치고 있다. 카카오페이는 선물하기 등 결제 기능이 있지만, 이커머스와 연동돼 있는 네이버, 카드사와 연동돼 있는 토스에 비해 시장점유율이 낮은 상황이다. 이 때문에 카카오페이는 이커머스 기능이 있는 쓱페이·스마일페이 인수를 시도하며 시장점유율 확대를 노릴 것으로 전망된다. 쓱페이와 스마일페이는 주 이용률 빈도는 낮지만, 이용자가 약 2500만명에 달해 카카오페이와 시너지 효과를 극대화할 수 있다. 앞서 신세계그룹은 지난해 한 차례 쓱페이·스마일페이에 대한 매각 작업을 추진하며 우선협상대상자로 토스(비바리퍼블리카)를 선정한 바 있다. 당시 거론된 가격은 7000억원 선이었다. 하지만 양측은 이후 시너지 창출 방안과 관련해 이견이 생겼고, 결국 매각 작업이 무산됐다. 이런 가운데 이날 이마트의 자회사 SSG닷컴은 간편결제 사업부인 \'쓱페이\'를 물적분할해 신설 회사인

In [38]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification
from pathlib import Path

model_id = "KPF/KPF-BERT-NER"
save_dir = Path("../../automoation/models/ner_onnx")

# ONNX로 export
model = ORTModelForTokenClassification.from_pretrained(model_id, export=True)
model.save_pretrained(save_dir)

# tokenizer도 저장
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(save_dir)

print("✅ NER ONNX 모델 저장 완료:", save_dir)

✅ NER ONNX 모델 저장 완료: ..\..\automoation\models\ner_onnx


In [ ]:
import onnxruntime as ort

def get_ner_tokenizer():
    """
    ONNX NER 모델을 위한 토크나이저를 반환합니다.
    """

    # 🟢 토크나이저 로딩
    tokenizer_ner = Tokenizer.from_file("../../automation/models/ner_onnx/tokenizer.json")

    # 🟢 ONNX 모델 세션 로딩
    session_ner = ort.InferenceSession("../../automation/models/ner_onnx/model.onnx")

    return tokenizer_ner, session_ner

In [67]:
import numpy as np
from label_map import id2label  # 라벨 ID ↔ 라벨명 매핑

tokenizer_ner, session_ner = get_ner_tokenizer()

def get_ner_tokens(tokenizer, session, text, id2label):
    # 🟡 토큰화 및 입력값 준비
    encoding = tokenizer.encode(text)
    input_ids = np.array([encoding.ids], dtype=np.int64)
    attention_mask = np.ones_like(input_ids, dtype=np.int64)
    token_type_ids = np.zeros_like(input_ids, dtype=np.int64)

    # 🔵 ONNX 추론 실행
    inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
    }

    logits = session.run(None, inputs)[0]  # shape: (1, seq_len, num_labels)

    # 🔵 라벨 인덱스 → 실제 라벨명
    preds = np.argmax(logits, axis=-1)[0]
    labels = [id2label[p] for p in preds[: len(encoding.tokens)]]

    # 🔵 시각화
    tokens = encoding.tokens

    return tokens, labels

In [ ]:
def extract_ogg_economy(tokens, labels, target_label="OGG_ECONOMY"):
    merged_words = []
    current_word = ""

    for token, label in zip(tokens, labels):
        token_clean = token.replace("##", "") if token.startswith("##") else token

        if label == f"B-{target_label}":
            if current_word:
                merged_words.append(current_word)
            current_word = token_clean

        elif label == f"I-{target_label}":
            current_word += token_clean

        else:
            if current_word:
                merged_words.append(current_word)
                current_word = ""

    if current_word:
        merged_words.append(current_word)

    stock_list = merged_words.copy()

    return stock_list

In [71]:
import numpy as np
from label_map import id2label  # 라벨 ID ↔ 라벨명 매핑

text = """
세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드론이 방한해 이창용 한국은행 총재, 이재용 삼성전자 회장, 김병주 MBK파트너스 회장 등과 함께 한국의 금융당국, 기업 및 금융계와의 협력 방안을 논의했다.
"""

tokens, labels = get_ner_tokens(tokenizer_ner, session_ner, text, id2label)
stock_list = extract_ogg_economy(tokens, labels, target_label="OGG_ECONOMY")
stock_list

['골드만삭스', '한국은행', '삼성전자', 'MBK파트너스']